In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip3 install thop

In [ ]:
root_path = '/content/drive/MyDrive/M5-FITS/processed-nonparam/'
checkpoints = '/content/drive/MyDrive/M5-thesis/M5-FITS/checkpoints'
fits_dir = '/content/drive/MyDrive/FITS'
thesis_dir = '/content/drive/MyDrive/M5-thesis'
processed_data_dir_base = '/content/drive/MyDrive/M5-A1/2. data/processed/'
BASE     = processed_data_dir_base+'grid_part_1.pkl'
PRICE    = processed_data_dir_base+'grid_part_2.pkl'
CALENDAR = processed_data_dir_base+'grid_part_3.pkl'

In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

os.chdir(thesis_dir)
from models.FITS import Model
# os.chdir(fits_dir + '/data_provider')
# from data_provider.data_factory import data_provider

In [ ]:
global_df = pd.concat([pd.read_pickle(BASE),
                pd.read_pickle(PRICE).iloc[:,2:],
                pd.read_pickle(CALENDAR).iloc[:,2:]],
                axis=1)

In [ ]:
stores = global_df['store_id'].unique()
depts = global_df['dept_id'].unique()

In [ ]:
import os
import numpy as np
import pandas as pd
import os
import torch
import random
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

class Config(object):
  def __init__(self, args):
    #basic config
    self.is_training = args.get('is_training', 1)
    self.model_id = args.get('model_id', 'test')
    self.model = args.get('model', 'Autoformer')
    #dataloader
    self.data = args.get('data', 'test')
    self.root_path = args.get('root_path', '/content/drive/MyDrive/M5-FITS/processed-nonparam')
    self.data_path = args.get('data_path', 'm5.csv')
    self.features = args.get('features', 'S')
    self.target = args.get('target', 'sales')
    self.freq = args.get('freq', 'd')
    self.checkpoints = args.get('checkpoints', '/content/drive/MyDrive/M5-FITS/checkpoints')
    #forecasting
    self.seq_len = args.get('seq_len', 56)
    self.pred_len = args.get('pred_len', 28)
    self.label_len = args.get('label_len', 28)
    self.individual = args.get('individual', False)
    #optimization
    self.num_workers = args.get('num_workers', 10)
    self.itr = args.get('itr', 2)
    self.train_epochs = args.get('train_epochs', 100)
    self.batch_size = args.get('batch_size', 32)
    self.patience = args.get('patience', 3)
    self.learning_rate = args.get('learning_rate', 0.0001)
    self.des = args.get('des', 'test')
    self.loss = args.get('loss', 'mse')
    self.lradj = args.get('lradj', 'type3')
    self.use_amp = args.get('use_amp', False)
    #GPU
    self.use_gpu = args.get('use_gpu', True)
    self.gpu = args.get('gpu', 0)
    self.use_multi_gpu = args.get('use_multi_gpu', False)
    self.devices = args.get('devices', '0,1,2,3')
    self.test_flop = args.get('test_flop', False)
    #Augmentation
    self.aug_method = args.get('aug_method', 'NA')
    self.aug_rate = args.get('aug_rate', 0.5)
    self.in_batch_augmentation = args.get('in_batch_augmentation', False)
    self.in_dataset_augmentation = args.get('in_dataset_augmentation', False)
    self.data_size = args.get('data_size', 1)
    self.aug_data_size = args.get('aug_data_size', 1)
    self.seed = args.get('seed', 2021)
    #continue learning
    self.testset_div = args.get('testset_div', 2)
    self.test_time_train = args.get('test_time_train', False)
    #Formers
    self.embed = args.get('embed', 'timeF')
    self.enc_in = args.get('enc_in', 7)
    self.dec_in = args.get('dec_in', 7)
    self.c_out = args.get('c_out', 7)
    self.d_model = args.get('d_model', 512)
    self.n_heads = args.get('n_heads', 8)
    self.e_layers = args.get('e_layers', 2)
    self.d_layers = args.get('d_layers', 1)
    self.d_ff = args.get('d_ff', 2048)
    self.moving_avg = args.get('moving_avg', 25)
    self.factor = args.get('factor', 1)
    self.distil = args.get('distil', True)
    self.dropout = args.get('dropout', 0.1)
    self.activation = args.get('activation', 'relu')
    self.output_attention = args.get('output_attention', False)
    self.do_predict = args.get('do_predict', False)

    #Flinear
    self.train_mode = args.get('train_mode', 0)
    self.cut_freq = args.get('cut_freq', 0)
    self.base_T = args.get('base_T', 24)
    self.H_order = args.get('H_order', 2)

    self.use_gpu = True if torch.cuda.is_available() and self.use_gpu else False
    cfreq = args.get('cut_freq', 0)
    if cfreq == 0:
      self.cut_freq = int(self.seq_len // self.base_T + 1) * self.H_order + 10

    fix_seed = self.seed
    random.seed(fix_seed)
    torch.manual_seed(fix_seed)
    np.random.seed(fix_seed)

In [ ]:
args = {
    'root_path': '/content/drive/MyDrive/M5-FITS/processed/v1',
    'data': 'custom',
    'features': 'MS',
    'model': 'FITS'
}
config = Config(args)

In [ ]:
from torch.utils.data import DataLoader
from utils.timefeatures import time_features

start_date = pd.Timestamp("2011-01-29")
TRAIN_START = 0
TRAIN_END = 1941
TEST_END = 1969
TRAIN_LEN = TRAIN_END - TRAIN_START
HORIZON = 28

"""
get df_train and features
"""
def prepare_features(calendar):
  df = calendar.copy()
  df['snap_CA'] = df['snap_CA'].astype(int)
  df['snap_TX'] = df['snap_TX'].astype(int)
  df['snap_WI'] = df['snap_WI'].astype(int)
  df['snap_count'] = df[['snap_CA','snap_TX','snap_WI']].apply(np.sum,axis=1)

  df['is_event_1'] = [isinstance(x , str)*1 for x in df['event_name_1']]
  df['is_event_2'] = [isinstance(x , str)*1 for x in df['event_name_2']]
  df['is_event'] = df[['is_event_1', 'is_event_2']].apply(np.sum, axis=1)
  #turn into boolean
  # df['is_event'] = np.where(df['is_event']>0,1,0)

  df.drop(columns=['event_name_1', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event_1', 'is_event_2'], inplace=True)
  df['date'] = start_date + pd.to_timedelta(df['d'] - 1, unit='D')


  return df

"""
get df_train and features
"""
def get_data_by_store_id(store_id, dept_id):
  df = global_df.copy()

  store_data = df[(df['store_id'] == store_id) & (df['dept_id'] == dept_id) & (df['d'] >= TRAIN_START)]
  # df_train = store_data.pivot(index='id', columns='d', values='sales')
  # df_train.fillna(0, inplace=True)
  # df_train = store_data.groupby(['d']).agg({
  #   'sales': 'sum',
  #   'event_name_1': 'first',
  #   'event_name_2': 'first',
  #   'snap_CA': 'first',
  #   'snap_TX': 'first',
  #   'snap_WI': 'first',
  #   'tm_w_end': 'first',
  #   'tm_dw': 'first',
  #   'tm_wm': 'first',
  #   'tm_m': 'first',
  #   'tm_w': 'first',
  #   'sell_price': 'mean',
  # }).reset_index()
  # df_calendar.index = df_calendar['d']

  # del df, store_data
  # gc.collect()

  # df_train = prepare_features(df_train)
  del df
  gc.collect()

  return store_data

def aggregate_data(df):
  df = df.groupby(['d']).agg({
    'sales': 'sum',
    'event_name_1': 'first',
    'event_name_2': 'first',
    'snap_CA': 'first',
    'snap_TX': 'first',
    'snap_WI': 'first',
    'tm_w_end': 'first',
    'tm_dw': 'first',
    'tm_wm': 'first',
    'tm_m': 'first',
    'tm_w': 'first',
    'sell_price': 'mean',
  }).reset_index()

  df = prepare_features(df)

  return df


class Dataset_Custom(Dataset):
    def __init__(self, config, df, flag='train', size=None,
                 features='S',
                 target='OT', scale=True, timeenc=0, freq='h'):
        self.args = config
        # info
        if size == None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq

        self.df_raw = df

        self.__read_data__()
        self.collect_all_data()
        if self.args.in_dataset_augmentation and self.set_type==0:
            self.data_augmentation()

    def __read_data__(self):
        self.scaler = StandardScaler()
        #TODO: read bottom-level data. Reproduce similar functionality as in DLinear

        '''
        df_raw.columns: ['date', ...(other features), target feature]
        '''
        cols = list(self.df_raw.columns)
        cols.remove(self.target)
        cols.remove('date')
        self.df_raw = self.df_raw[['date'] + cols + [self.target]]
        # print(cols)
        # num_train = int(len(df_raw) * 0.7)
        # num_test = int(len(df_raw) * 0.2)
        num_test = HORIZON  # Fixed to the last 28 days
        num_train = int((TRAIN_END - TRAIN_START) * 0.8)
        num_vali = TRAIN_END - TRAIN_START - num_train - num_test
        border1s = [0, num_train - self.seq_len, len(self.df_raw) - num_test - self.seq_len]
        border2s = [num_train, num_train + num_vali, len(self.df_raw)]

        if self.args.test_time_train:
            num_train = int(len(self.df_raw) * 0.9)
            border1s = [0, num_train - self.seq_len, len(self.df_raw)]
            border2s = [num_train, len(self.df_raw), len(self.df_raw)]

        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        if self.features == 'M' or self.features == 'MS':
            cols_data = self.df_raw.columns[1:]
            df_data = self.df_raw[cols_data]
        elif self.features == 'S':
            df_data = self.df_raw[[self.target]]

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            # print(self.scaler.mean_)
            # exit()
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values

        df_stamp = self.df_raw[['date']][border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['date'], 1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)

        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        self.data_stamp = data_stamp
        print(border1, border2)

    def regenerate_augmentation_data(self):
        self.collect_all_data()
        self.data_augmentation()

    def reload_data(self, x_data, y_data, x_time, y_time):
        self.x_data = x_data
        self.y_data = y_data
        self.x_time = x_time
        self.y_time = y_time

    def collect_all_data(self):
        self.x_data = []
        self.y_data = []
        self.x_time = []
        self.y_time = []
        data_len = len(self.data_x) - self.seq_len - self.pred_len + 1
        mask_data_len = int((1-self.args.data_size) * data_len) if self.args.data_size < 1 else 0
        for i in range(len(self.data_x) - self.seq_len - self.pred_len + 1):
            if (self.set_type == 0 and i >= mask_data_len) or self.set_type != 0:
                s_begin = i
                s_end = s_begin + self.seq_len
                r_begin = s_end - self.label_len
                r_end = r_begin + self.label_len + self.pred_len
                self.x_data.append(self.data_x[s_begin:s_end])
                self.y_data.append(self.data_y[r_begin:r_end])
                self.x_time.append(self.data_stamp[s_begin:s_end])
                self.y_time.append(self.data_stamp[r_begin:r_end])

    def data_augmentation(self):
        origin_len = len(self.x_data)
        if not self.args.closer_data_aug_more:
            aug_size = [self.args.aug_data_size for i in range(origin_len)]
        else:
            aug_size = [int(self.args.aug_data_size * i/origin_len) + 1 for i in range(origin_len)]

        for i in range(origin_len):
            for _ in range(aug_size[i]):
                aug = augmentation('dataset')
                if self.args.aug_method == 'f_mask':
                    x,y = aug.freq_dropout(self.x_data[i],self.y_data[i],dropout_rate=self.args.aug_rate)
                elif self.args.aug_method == 'f_mix':
                    rand = float(np.random.random(1))
                    i2 = int(rand*len(self.x_data))
                    x,y = aug.freq_mix(self.x_data[i],self.y_data[i],self.x_data[i2],self.y_data[i2],dropout_rate=self.args.aug_rate)
                else:
                    raise ValueError
                self.x_data.append(x)
                self.y_data.append(y)
                self.x_time.append(self.x_time[i])
                self.y_time.append(self.y_time[i])

    def __getitem__(self, index):
        seq_x = self.x_data[index]
        seq_y = self.y_data[index]
        return seq_x, seq_y, self.x_time[index], self.y_time[index]

    def __len__(self):
        return len(self.x_data)

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

def data_provider(args, df, flag):
  timeenc = 0 if args.embed != 'timeF' else 1

  if flag == 'test':
      shuffle_flag = False
      drop_last = False # True
      batch_size = args.batch_size
      freq = args.freq
  else:
      shuffle_flag = True
      drop_last = True
      batch_size = args.batch_size
      freq = args.freq

  data_set = Dataset_Custom(
      config=args,
      flag=flag,
      df=df,
      size=[args.seq_len, args.label_len, args.pred_len],
      features=args.features,
      target=args.target,
      timeenc=timeenc,
      freq=freq,
      scale=True #try with True later
  )

  data_loader = DataLoader(
      data_set,
      batch_size=batch_size,
      shuffle=shuffle_flag,
      num_workers=args.num_workers,
      drop_last=drop_last)

  return data_set, data_loader

In [ ]:
from models.FITS import Model
from utils.tools import EarlyStopping, adjust_learning_rate, test_params_flop, visual
from utils.metrics import metric
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from utils.augmentations import augmentation
import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

from thop import profile

class FITS(object):
    def __init__(self, args, df):
        self.args = args
        self.device = self._acquire_device()
        self.model = self._build_model().to(self.device)
        self.df_train = df
        self.top_level = self.get_top_level()

    def get_top_level(self):
      """
      Calculate top level time series by aggregating low level time-series
      """
      data = aggregate_data(self.df_train)

      return data

    def get_weights(self):
      temp = self.df_train.pivot(index='id', columns='d', values='sales')
      temp.fillna(0, inplace=True)
      w = np.sum(temp.iloc[:, -56:-28].values, axis=1) / sum(self.top_level['sales'].iloc[-56:-28])
      w = w.reshape(len(w), 1)

      del temp
      gc.collect()

      return w

    def _build_model(self):
        raise NotImplementedError
        return None

    def _acquire_device(self):
        if self.args.use_gpu:
            os.environ["CUDA_VISIBLE_DEVICES"] = str(
                self.args.gpu) if not self.args.use_multi_gpu else self.args.devices
            device = torch.device('cuda:{}'.format(self.args.gpu))
            print('Use GPU: cuda:{}'.format(self.args.gpu))
        else:
            device = torch.device('cpu')
            print('Use CPU')
        return device

    def _get_data(self):
        pass

    def vali(self):
        pass

    def train(self):
        pass

    def test(self):
        pass

class M5FITS(FITS):
    def __init__(self, args, df):
        super(M5FITS, self).__init__(args, df)

    def _build_model(self):
        model = Model(self.args).float()

        if self.args.use_multi_gpu and self.args.use_gpu:
            model = nn.DataParallel(model, device_ids=self.args.device_ids)

        return model

    def _get_data(self, flag):
        data_set, data_loader = data_provider(self.args, self.get_top_level(), flag)

        return data_set, data_loader

    def _select_optimizer(self):
        model_optim = optim.Adam(self.model.parameters(), lr=self.args.learning_rate)
        print('!!!!!!!!!!!!!!learning rate!!!!!!!!!!!!!!!')
        print(self.args.learning_rate)
        return model_optim

    def _select_criterion(self):
        criterion = nn.MSELoss()
        return criterion

    def _get_profile(self, model):
        _input=torch.randn(self.args.batch_size, self.args.seq_len, self.args.enc_in).to(self.device)
        macs, params = profile(model, inputs=(_input,))
        print('FLOPs: ', macs)
        print('params: ', params)
        return macs, params

    def vali(self, vali_data, vali_loader, criterion):
        total_loss = []
        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(vali_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float().to(self.device)[:,-self.args.pred_len:,:]
                batch_xy = torch.cat([batch_x, batch_y], dim=1)

                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
                # encoder - decoder
                if 'FITS' in self.args.model:
                    outputs, low = self.model(batch_x)
                elif 'SCINet' in self.args.model:
                    outputs = self.model(batch_x)
                else:
                    if self.args.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                f_dim = -1 if self.args.features == 'MS' else 0
                outputs = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:]

                pred = outputs.detach().cpu()
                true = batch_y.detach().cpu()

                loss = criterion(pred, true)

                total_loss.append(loss)
        total_loss = np.average(total_loss)
        self.model.train()
        return total_loss

    def train(self, setting, ft=False):
        train_data, train_loader = self._get_data(flag='train')
        vali_data, vali_loader = self._get_data(flag='val')
        test_data, test_loader = self._get_data(flag='test')
        print(self.model)
        self._get_profile(self.model)
        print('Trainable parameters: ', sum(p.numel() for p in self.model.parameters() if p.requires_grad))

        path = os.path.join(self.args.checkpoints, setting)
        if not os.path.exists(path):
            os.makedirs(path)

        time_now = time.time()

        train_steps = len(train_loader)
        early_stopping = EarlyStopping(patience=self.args.patience, verbose=True)

        model_optim = self._select_optimizer()
        criterion = self._select_criterion()

        for epoch in range(self.args.train_epochs):
            iter_count = 0
            train_loss = []

            self.model.train()
            epoch_time = time.time()
            if self.args.in_dataset_augmentation:
                train_loader.dataset.regenerate_augmentation_data()

            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_loader):
                iter_count += 1
                model_optim.zero_grad()

                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float().to(self.device)[:,-self.args.pred_len:,:]
                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)
                # print(batch_x.shape, batch_y.shape)
                batch_xy = torch.cat([batch_x, batch_y], dim=1)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)

                # encoder - decoder
                if 'FITS' in self.args.model:
                        outputs, low = self.model(batch_x)
                elif 'SCINet' in self.args.model:
                        outputs = self.model(batch_x)
                else:
                    if self.args.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark, batch_y)

                # print(outputs.shape,batch_y.shape)
                f_dim = -1 if self.args.features == 'MS' else 0
                if ft:
                    outputs = outputs[:, -self.args.pred_len:, f_dim:]
                    batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
                    # print(outputs.shape,batch_xy.shape)
                    #loss = criterion(outputs, batch_xy)
                    loss = criterion(outputs, batch_y)
                else:
                    outputs = outputs[:, :, f_dim:]
                    # batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device) #???
                    loss = criterion(outputs, batch_xy)
                train_loss.append(loss.item())

                if (i + 1) % 100 == 0:
                    print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                    speed = (time.time() - time_now) / iter_count
                    left_time = speed * ((self.args.train_epochs - epoch) * train_steps - i)
                    print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()

                loss.backward()
                model_optim.step()

            print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
            train_loss = np.average(train_loss)
            vali_loss = self.vali(vali_data, vali_loader, criterion)
            test_loss = self.vali(test_data, test_loader, criterion)

            print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
                epoch + 1, train_steps, train_loss, vali_loss, test_loss))
            early_stopping(vali_loss, self.model, path)
            if early_stopping.early_stop:
                print("Early stopping")
                break

            adjust_learning_rate(model_optim, epoch + 1, self.args)

        best_model_path = path + '/' + 'checkpoint.pth'
        self.model.load_state_dict(torch.load(best_model_path))

        return self.model

    def predict_top_level(self, test=0):
      pred_set, pred_loader = self._get_data(flag='test')
      if test:
        print('loading model')
        self.model.load_state_dict(torch.load(os.path.join('/content/drive/MyDrive/M5-FITS/checkpoints/' + setting, 'checkpoint.pth')))

      preds = []

      self.model.eval()
      with torch.no_grad():
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(pred_loader):
          batch_x = batch_x.float().to(self.device)
          batch_y = batch_y.float()
          batch_x_mark = batch_x_mark.float().to(self.device)
          batch_y_mark = batch_y_mark.float().to(self.device)

          # decoder input
          dec_inp = torch.zeros([batch_y.shape[0], self.args.pred_len, batch_y.shape[2]]).float().to(batch_y.device)
          dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
          # encoder - decoder
          outputs, low = self.model(batch_x)
          f_dim = -1 if self.args.features == 'MS' else 0
          outputs_ = outputs[:, -self.args.pred_len:, f_dim:]
          pred = outputs_.detach().cpu().numpy()  # .squeeze()
          preds.append(pred)

        preds = np.array(preds)
        preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
        preds = pred_set.inverse_transform(preds[0])

      return preds.flatten()

    def predict_bottom_level(self):
      w = self.get_weights()
      top_level_preds = self.predict_top_level()
      top_level_preds = top_level_preds.reshape(1,len(top_level_preds))
      preds = np.multiply(top_level_preds, w)

      item_ids = self.df_train['id'].unique()
      res = pd.DataFrame({
          'id': item_ids
      })

      predictions_df = pd.DataFrame(preds, columns=[f'F{i+1}' for i in range(28)])
      preds = pd.concat([res['id'], predictions_df], axis=1)

      preds.set_index('id', inplace=True)

      return preds

    def test(self, setting, test=0):
        test_data, test_loader = self._get_data(flag='test')

        if test:
            print('loading model')
            self.model.load_state_dict(torch.load(os.path.join('./checkpoints/' + setting, 'checkpoint.pth')))

        preds = []
        trues = []
        inputx = []
        reconx = []
        inputxy = []
        reconxy = []
        lows = []
        folder_path = './test_results/' + setting + '/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        self.model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
                batch_x = batch_x.float().to(self.device)
                batch_y = batch_y.float().to(self.device)[:,-self.args.pred_len:,:]
                batch_xy = torch.cat([batch_x, batch_y], dim=1).float().to(self.device)

                batch_x_mark = batch_x_mark.float().to(self.device)
                batch_y_mark = batch_y_mark.float().to(self.device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -self.args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :self.args.label_len, :], dec_inp], dim=1).float().to(self.device)
                # encoder - decoder

                if 'FITS' in self.args.model:
                        outputs, low = self.model(batch_x)
                elif 'SCINet' in self.args.model:
                        outputs = self.model(batch_x)
                else:
                    if self.args.output_attention:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]

                    else:
                        outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                f_dim = -1 if self.args.features == 'MS' else 0
                # print(outputs.shape,batch_y.shape)
                outputs_ = outputs[:, -self.args.pred_len:, f_dim:]
                batch_y = batch_y[:, -self.args.pred_len:, f_dim:].to(self.device)
                outputs_ = outputs_.detach().cpu().numpy()
                batch_y = batch_y.detach().cpu().numpy()


                pred = outputs_  # outputs.detach().cpu().numpy()  # .squeeze()
                true = batch_y  # batch_y.detach().cpu().numpy()  # .squeeze()

                preds.append(pred)
                trues.append(true)
                inputx.append(batch_x.detach().cpu().numpy())
                inputxy.append(batch_xy.detach().cpu().numpy())
                reconx.append(outputs[:, :-self.args.pred_len, f_dim:].detach().cpu().numpy())
                reconxy.append(outputs.detach().cpu().numpy())
                lows.append(low.detach().cpu().numpy())
                if i % 20 == 0:
                    input = batch_x.detach().cpu().numpy()
                    gt = np.concatenate((input[0, :, -1], true[0, :, -1]), axis=0)
                    pd = np.concatenate((input[0, :, -1], pred[0, :, -1]), axis=0)
                    visual(gt, pd, os.path.join(folder_path, str(i) + '.pdf'))

        if self.args.test_flop:
            test_params_flop((batch_x.shape[1],batch_x.shape[2]))
            exit()
        preds = np.concatenate(preds, axis=0)
        trues = np.concatenate(trues, axis=0)
        # inputx = np.array(inputx)
        # reconx = np.array(reconx)
        # reconxy = np.array(reconxy)
        # inputxy = np.array(inputxy)
        # lows = np.array(lows)


        # preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
        # trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
        # inputx = inputx.reshape(-1, inputx.shape[-2], inputx.shape[-1])
        # reconx = reconx.reshape(-1, reconx.shape[-2], reconx.shape[-1])
        # reconxy = reconxy.reshape(-1, reconxy.shape[-2], reconxy.shape[-1])
        # inputxy = inputxy.reshape(-1, inputxy.shape[-2], inputxy.shape[-1])
        # lows = lows.reshape(-1, lows.shape[-2], lows.shape[-1])

        # try:
        #     for i in range(0,2800,300):

        #         # create a figure with 3 subplots
        #         fig, axs = plt.subplots(3, 1, figsize=(10, 10))
        #         # plot pred and true in the first subplot
        #         axs[0].plot(trues[i, :, -1], label='true')
        #         axs[0].plot(preds[i, :, -1], label='pred')
        #         axs[0].set_title('pred and true')
        #         # plot inputx and reconx in the second subplot
        #         axs[1].plot(inputx[i, :, -1], label='inputx')
        #         axs[1].plot(reconx[i, :, -1], label='reconx')
        #         axs[1].set_title('inputx and reconx')
        #         # plot inputxy and reconxy in the third subplot
        #         axs[2].plot(inputxy[i, :, -1], label='inputxy')
        #         axs[2].plot(reconxy[i, :, -1], label='reconxy')
        #         axs[2].plot(lows[i, :, -1])
        #         axs[2].set_title('inputxy and reconxy')
        #         # show the legend
        #         plt.legend()
        #         # save the figure to file
        #         fig.savefig(os.path.join(folder_path, str(i) + '_F.png'))
        #         # print('plottting')
        # except:
        #     pass

        # result save
        # folder_path = './results/' + setting + '/'
        # if not os.path.exists(folder_path):
        #     os.makedirs(folder_path)

        # mae, mse, rmse, mape, mspe, rse, corr = metric(preds, trues)
        # print('mse:{}, mae:{}, rse:{}, corr:{}'.format(mse, mae, rse, corr))
        # f = open("result.txt", 'a')
        # f.write(setting + "  \n")
        # f.write('mse:{}, mae:{}, rse:{}, corr:{}'.format(mse, mae, rse, corr))
        # f.write('\n')
        # f.write('\n')
        # f.close()

        # np.save(folder_path + 'metrics.npy', np.array([mae, mse, rmse, mape, mspe,rse, corr]))
        # np.save(folder_path + 'pred.npy', preds)
        # np.save(folder_path + 'true.npy', trues)
        # np.save(folder_path + 'x.npy', inputx)
        return

In [ ]:
setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_H{}_{}'.format(
            config.model_id,
            config.model,
            config.data,
            config.features,
            config.seq_len,
            config.label_len,
            config.pred_len,
            config.H_order, 1)

config.features = 'S'
config.seq_len = 42
df = get_data_by_store_id('CA_1', 'HOBBIES_1')
main = M5FITS(config, df=df)

main.train(setting, ft=True)


Use CPU
0 1552
1510 1913
1899 1969
Model(
  (freq_upsampler): Linear(in_features=16, out_features=26, bias=True)
)
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
FLOPs:  93184.0
params:  442.0
Trainable parameters:  442
!!!!!!!!!!!!!!learning rate!!!!!!!!!!!!!!!
0.0001
Epoch: 1 cost time: 4.765042304992676
Epoch: 1, Steps: 46 | Train Loss: 1.2755695 Vali Loss: 1.0650271 Test Loss: 17.9731922
Validation loss decreased (inf --> 1.065027).  Saving model ...
Updating learning rate to 0.0001
Epoch: 2 cost time: 4.285555601119995
Epoch: 2, Steps: 46 | Train Loss: 1.2384347 Vali Loss: 1.0352753 Test Loss: 17.9870434
Validation loss decreased (1.065027 --> 1.035275).  Saving model ...
Updating learning rate to 9.5e-05
Epoch: 3 cost time: 3.530832529067993
Epoch: 3, Steps: 46 | Train Loss: 1.2051006 Vali Loss: 1.0026989 Test Loss: 18.0012722
Validation loss decreased (1.035275 --> 1.002699).  Saving model ...
Updating learning rate to 9.025e-05
Epoch: 4 cost time: 

Model(
  (freq_upsampler): Linear(in_features=16, out_features=26, bias=True)
)

In [ ]:
p = main.predict_bottom_level()
p

1899 1969


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_008_CA_1_evaluation,11.183013,8.719904,8.357412,10.838589,11.378608,10.056700,10.553419,12.306272,12.464529,10.626851,...,11.913912,11.741090,10.854269,10.811574,10.506216,9.305079,9.915589,11.475986,11.059407,11.310079
HOBBIES_1_009_CA_1_evaluation,0.655682,0.511265,0.490011,0.635487,0.667150,0.589644,0.618767,0.721540,0.730819,0.623073,...,0.698536,0.688403,0.636407,0.633903,0.616000,0.545575,0.581370,0.672859,0.648434,0.663132
HOBBIES_1_010_CA_1_evaluation,0.546401,0.426054,0.408343,0.529573,0.555958,0.491370,0.515639,0.601284,0.609016,0.519227,...,0.582113,0.573669,0.530339,0.528253,0.513333,0.454646,0.484475,0.560716,0.540362,0.552610
HOBBIES_1_012_CA_1_evaluation,0.254987,0.198825,0.190560,0.247134,0.259447,0.229306,0.240632,0.280599,0.284208,0.242306,...,0.271653,0.267712,0.247491,0.246518,0.239555,0.212168,0.226088,0.261667,0.252169,0.257885
HOBBIES_1_015_CA_1_evaluation,2.586299,2.016655,1.932822,2.506644,2.631535,2.325817,2.440693,2.846076,2.882676,2.457676,...,2.755335,2.715366,2.510271,2.500397,2.429776,2.151989,2.293182,2.654055,2.557713,2.615686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOBBIES_1_125_CA_1_evaluation,0.546401,0.426054,0.408343,0.529573,0.555958,0.491370,0.515639,0.601284,0.609016,0.519227,...,0.582113,0.573669,0.530339,0.528253,0.513333,0.454646,0.484475,0.560716,0.540362,0.552610
HOBBIES_1_232_CA_1_evaluation,2.039898,1.590601,1.524479,1.977072,2.075577,1.834447,1.925054,2.244792,2.273660,1.938448,...,2.173222,2.141697,1.979932,1.972144,1.916443,1.697343,1.808707,2.093340,2.017351,2.063076
HOBBIES_1_269_CA_1_evaluation,0.109280,0.085211,0.081669,0.105915,0.111192,0.098274,0.103128,0.120257,0.121803,0.103845,...,0.116423,0.114734,0.106068,0.105651,0.102667,0.090929,0.096895,0.112143,0.108072,0.110522


In [ ]:
def train_model(model_name, args):
  submission = pd.read_csv('/content/drive/MyDrive/data/m5-data/sample_submission.csv')
  submission.set_index('id', inplace=True)
  config = Config(args)

  for store_id in stores:
    for dept_id in depts:
      print(f"training {store_id} + {dept_id}")
      setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_H{}_{}_{}_{}'.format(
            config.model_id,
            config.model,
            config.data,
            config.features,
            config.seq_len,
            config.label_len,
            config.pred_len,
            config.H_order,
            dept_id,
            store_id, 1)

      df_train = get_data_by_store_id(store_id, dept_id)
      model = M5FITS(config, df=df_train)
      model.train(setting, ft=True)
      sub_submission = model.predict_bottom_level()

      submission.update(sub_submission)

      del df_train
      del model
      gc.collect()

  submission.reset_index(inplace=True)
  submission.to_csv(f"/content/drive/MyDrive/data/m5-data/submissions/{model_name}.csv", index=False)

In [ ]:
models = [
  {
    'data': 'custom',
    'features': 'S',
    'model': 'FITS',
    'seq_len': 56,
  },
  {
    'data': 'custom',
    'features': 'S',
    'model': 'FITS',
    'seq_len': 42,
  },
  {
    'data': 'custom',
    'features': 'S',
    'model': 'FITS',
    'seq_len': 30,
  },
  {
    'data': 'custom',
    'features': 'S',
    'model': 'FITS',
    'seq_len': 84,
  },
]

In [ ]:
for model in models:
  model_name = f"fits-lb-{model['seq_len']}-epochs-100-v1"
  submission = pd.read_csv('/content/drive/MyDrive/data/m5-data/sample_submission.csv')
  submission.set_index('id', inplace=True)
  config = Config(model)

  for store_id in stores:
    for dept_id in depts:
      print(f"training {store_id} + {dept_id}")
      setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_H{}_{}_{}_{}'.format(
            config.model_id,
            config.model,
            config.data,
            config.features,
            config.seq_len,
            config.label_len,
            config.pred_len,
            config.H_order,
            dept_id,
            store_id, 1)

      df_train = get_data_by_store_id(store_id, dept_id)
      model = M5FITS(config, df=df_train)
      model.train(setting, ft=True)
      sub_submission = model.predict_bottom_level()

      submission.update(sub_submission)

      gc.collect()

  submission.reset_index(inplace=True)
  submission.to_csv(f"/content/drive/MyDrive/data/m5-data/submissions/{model_name}.csv", index=False)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/data/m5-data/sample_submission.csv')
submission.set_index('id', inplace=True)

In [ ]:
submission.update(p)

In [ ]:
submission

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_004_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_005_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FOODS_3_824_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FOODS_3_825_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
p

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_008_CA_1_evaluation,11.183013,8.719904,8.357412,10.838589,11.378608,10.056700,10.553419,12.306272,12.464529,10.626851,...,11.913912,11.741090,10.854269,10.811574,10.506216,9.305079,9.915589,11.475986,11.059407,11.310079
HOBBIES_1_009_CA_1_evaluation,0.655682,0.511265,0.490011,0.635487,0.667150,0.589644,0.618767,0.721540,0.730819,0.623073,...,0.698536,0.688403,0.636407,0.633903,0.616000,0.545575,0.581370,0.672859,0.648434,0.663132
HOBBIES_1_010_CA_1_evaluation,0.546401,0.426054,0.408343,0.529573,0.555958,0.491370,0.515639,0.601284,0.609016,0.519227,...,0.582113,0.573669,0.530339,0.528253,0.513333,0.454646,0.484475,0.560716,0.540362,0.552610
HOBBIES_1_012_CA_1_evaluation,0.254987,0.198825,0.190560,0.247134,0.259447,0.229306,0.240632,0.280599,0.284208,0.242306,...,0.271653,0.267712,0.247491,0.246518,0.239555,0.212168,0.226088,0.261667,0.252169,0.257885
HOBBIES_1_015_CA_1_evaluation,2.586299,2.016655,1.932822,2.506644,2.631535,2.325817,2.440693,2.846076,2.882676,2.457676,...,2.755335,2.715366,2.510271,2.500397,2.429776,2.151989,2.293182,2.654055,2.557713,2.615686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOBBIES_1_125_CA_1_evaluation,0.546401,0.426054,0.408343,0.529573,0.555958,0.491370,0.515639,0.601284,0.609016,0.519227,...,0.582113,0.573669,0.530339,0.528253,0.513333,0.454646,0.484475,0.560716,0.540362,0.552610
HOBBIES_1_232_CA_1_evaluation,2.039898,1.590601,1.524479,1.977072,2.075577,1.834447,1.925054,2.244792,2.273660,1.938448,...,2.173222,2.141697,1.979932,1.972144,1.916443,1.697343,1.808707,2.093340,2.017351,2.063076
HOBBIES_1_269_CA_1_evaluation,0.109280,0.085211,0.081669,0.105915,0.111192,0.098274,0.103128,0.120257,0.121803,0.103845,...,0.116423,0.114734,0.106068,0.105651,0.102667,0.090929,0.096895,0.112143,0.108072,0.110522


In [ ]:
np.sum(p.index.duplicated())

0

In [ ]:
model.predict_top_level()

1885 1969


ValueError: non-broadcastable output operand with shape (28,1) doesn't match the broadcast shape (28,10)

In [ ]:
model.df_train

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,...,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
161,HOBBIES_2_001_CA_1_evaluation,HOBBIES_2_001,HOBBIES_2,HOBBIES,CA_1,CA,1,1.0,0,5.468750,...,0,0,0,29,4,1,0,5,5,1
162,HOBBIES_2_002_CA_1_evaluation,HOBBIES_2_002,HOBBIES_2,HOBBIES,CA_1,CA,1,4.0,0,1.969727,...,0,0,0,29,4,1,0,5,5,1
163,HOBBIES_2_004_CA_1_evaluation,HOBBIES_2_004,HOBBIES_2,HOBBIES,CA_1,CA,1,0.0,0,2.470703,...,0,0,0,29,4,1,0,5,5,1
164,HOBBIES_2_006_CA_1_evaluation,HOBBIES_2_006,HOBBIES_2,HOBBIES,CA_1,CA,1,0.0,0,3.470703,...,0,0,0,29,4,1,0,5,5,1
165,HOBBIES_2_007_CA_1_evaluation,HOBBIES_2_007,HOBBIES_2,HOBBIES,CA_1,CA,1,0.0,0,0.970215,...,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47705467,HOBBIES_2_145_CA_1_evaluation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_1,CA,1969,NaN,3,2.880859,...,0,0,0,19,24,6,5,3,6,1
47705468,HOBBIES_2_146_CA_1_evaluation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_1,CA,1969,NaN,1,1.969727,...,0,0,0,19,24,6,5,3,6,1
47705469,HOBBIES_2_147_CA_1_evaluation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_1,CA,1969,NaN,212,0.970215,...,0,0,0,19,24,6,5,3,6,1
47705470,HOBBIES_2_148_CA_1_evaluation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_1,CA,1969,NaN,1,0.879883,...,0,0,0,19,24,6,5,3,6,1
